導入套件

In [4]:
import numpy as np
import matplotlib.pyplot as plt  
import pandas as pd
%matplotlib inline

載入資料_訓練集：2014 年 ~ 2018 年的 TSMC（共 1231 天）

In [5]:
dataset_train = pd.read_csv("台指數/台灣加權指數2014-2018.csv")

#將資料顯示為(資料長度,1) 等同reshape(-1,1)
training_set = dataset_train.iloc[: ,4:5].values
#training_set = dataset_train.iloc[: ,1:2].values
training_set

array([[8612.540039],
       [8546.540039],
       [8500.009766],
       ...,
       [9478.990234],
       [9641.55957 ],
       [9727.410156]])

In [6]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler

#做 Normalization，將資料壓縮在 [0,1] 之間
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [7]:
X_train = []   #預測點的前 60 天的資料
y_train = []   #預測點
for i in range(60, 1231):  # 1231 是訓練集總數 / 2014~2018總天數
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)  # 轉成numpy array的格式，以利輸入 RNN
print(X_train, y_train)

[[0.31284725 0.29567214 0.28356362 ... 0.38907094 0.38467043 0.38152176]
 [0.29567214 0.28356362 0.28676185 ... 0.38467043 0.38152176 0.38459496]
 [0.28356362 0.28676185 0.29813644 ... 0.38152176 0.38459496 0.39560793]
 ...
 [0.91321874 0.89872673 0.86098566 ... 0.58976462 0.58014403 0.55083956]
 [0.89872673 0.86098566 0.80847406 ... 0.58014403 0.55083956 0.53832265]
 [0.86098566 0.80847406 0.79255054 ... 0.55083956 0.53832265 0.58062789]] [0.38459496 0.39560793 0.40016956 ... 0.53832265 0.58062789 0.60296869]


因為現在 X_train 是 2-dimension，將它 reshape 成 3-dimension: [stock prices, timesteps, indicators]

In [8]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [9]:
# Import the Keras libraries and packages
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout,BatchNormalization

# Initialising the RNN
keras.backend.clear_session()
regressor = Sequential()

# 搭建 LSTM layer:

units: 神經元的數目 (setting 50)

第一層的 LSTM Layer 記得要設定input_shape參數

In [10]:
input_shape = (X_train.shape[1], 1)
print(input_shape)

(60, 1)


搭配使用dropout，這裡設為 0.2(Float between 0 and 1)

由於這邊的第四層 LSTM Layer 即將跟 Ouput Layer 做連接，因此注意這邊的 return_sequences 設為預設值 False （也就是不用寫上 return_sequences）

用了 4層 LSTM

In [11]:
# Adding the first LSTM layer and some Dropout regularisation,activation use relu functio
regressor.add(LSTM(units = 50,activation="relu", return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50,activation="relu", return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, activation="relu",return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, activation="relu"))
regressor.add(Dropout(0.2))

In [12]:
# Adding the output layer setting 1
regressor.add(Dense(units = 1))

# Compiling & Fitting LSTM model
optimizer: 選擇 Adam \\\
loss: 使用 MSE \\\
learning rate(lr) :0.001 \\\
epochs :100 \\\
batch size:16

In [ ]:
# Compiling
optimizer = keras.optimizers.Adam(lr=0.001)
regressor.compile(optimizer = optimizer, loss = 'mean_squared_error')

# 進行訓練
regressor.fit(X_train, y_train, epochs = 100, batch_size = 16)

Epoch 1/100
74/74 [==============================] - 18s 247ms/step - loss: nan
Epoch 2/100
74/74 [==============================] - 19s 258ms/step - loss: nan
Epoch 3/100
74/74 [==============================] - 19s 262ms/step - loss: nan
Epoch 4/100
74/74 [==============================] - 19s 258ms/step - loss: nan
Epoch 5/100
74/74 [==============================] - 19s 257ms/step - loss: nan
Epoch 6/100
74/74 [==============================] - 19s 262ms/step - loss: nan
Epoch 7/100
74/74 [==============================] - 19s 260ms/step - loss: nan
Epoch 8/100
74/74 [==============================] - 19s 258ms/step - loss: nan
Epoch 9/100
74/74 [==============================] - 19s 262ms/step - loss: nan
Epoch 10/100
74/74 [==============================] - 19s 258ms/step - loss: nan
Epoch 11/100
74/74 [==============================] - 19s 261ms/step - loss: nan
Epoch 12/100
74/74 [==============================] - 19s 258ms/step - loss: nan
Epoch 13/100
74/74 [=================

# 預測步驟

取測試集中 2019 年的股票資料（真實）

In [ ]:
dataset_test = pd.read_csv('台指數/台灣加權指數2019-2020.csv', usecols=[4], engine='python')
dataset_test.values

In [ ]:
dataset_test = pd.read_csv('台指數/台灣加權指數2019-2020.csv')
#real_stock_price = dataset_test.iloc[:, 4:5].values
real_stock_price = dataset_test.iloc[:, 1:2].values

取模型所預測的 2019 年股票資料（預測）

In [ ]:
dataset_total = pd.concat((dataset_train['Close'], dataset_test['Close']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs) # Feature Scaling

X_test = []
for i in range(60, 80):  # timesteps一樣60； 80 = 先前的60天資料+2017年的20天資料
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))  # Reshape 成 3-dimension

start predict

In [ ]:
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)  # to get the original scale

In [ ]:
# Visualising the results
plt.plot(real_stock_price, color = 'red', label = 'Real TSEC Stock Price')  # 紅線表示真實股價
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted TSEC Stock Price')  # 藍線表示預測股價
plt.title('TSEC Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('TSEC Stock Price')
plt.legend()
plt.show()